In [22]:
import sqlalchemy
sqlalchemy.__version__

'1.2.1'

In [23]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///Exc_180710_2.db", echo = True)
print(engine)

Engine(sqlite:///Exc_180710_2.db)


In [24]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
meta = MetaData()

In [25]:
users = Table('users', meta,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname',String),
             );

addresses = Table('addresses', meta,
                  Column('id',Integer, primary_key=True),
                  Column('user_id', None, ForeignKey('users.id')),
                  Column('email_address',String, nullable=False)
                  
                 );
meta.create_all(engine)

2018-07-10 13:46:46,163 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 13:46:46,164 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:46:46,165 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 13:46:46,165 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:46:46,166 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-10 13:46:46,167 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:46:46,168 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-07-10 13:46:46,168 INFO sqlalchemy.engine.base.Engine ()


In [26]:
for row in meta.tables:
    print(row)

users
addresses


In [27]:
print(meta)

MetaData(bind=None)


In [28]:
print(users.insert())

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [34]:
insert = users.insert().values(name = 'kim', fullname = 'Anonymous, Kim')
print(insert)

insert.compile().params

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


{'fullname': 'Anonymous, Kim', 'name': 'kim'}

In [48]:
insert.bind = engine
str(insert)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [46]:
conn = engine.connect()
conn

In [49]:
result = conn.execute(insert)
result.inserted_primary_key

2018-07-10 14:30:17,694 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 14:30:17,695 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2018-07-10 14:30:17,696 INFO sqlalchemy.engine.base.Engine COMMIT


[3]

In [50]:
insert = users.insert()
result = conn.execute(insert, name='Lee', fullname='Unknown, Lee')
result.inserted_primary_key

2018-07-10 14:31:08,946 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 14:31:08,947 INFO sqlalchemy.engine.base.Engine ('Lee', 'Unknown, Lee')
2018-07-10 14:31:08,948 INFO sqlalchemy.engine.base.Engine COMMIT


[4]

In [51]:
conn.execute(addresses.insert(), [
    {'user_id':1, 'email_address':'anonymous.kim@test.com'},
    {'user_id':2, 'email_address':'unknow.lee@test.com'}  
])

2018-07-10 14:32:18,114 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2018-07-10 14:32:18,115 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test.com'), (2, 'unknow.lee@test.com'))
2018-07-10 14:32:18,116 INFO sqlalchemy.engine.base.Engine COMMIT


In [53]:
from sqlalchemy.sql import select

In [54]:
query = select([users])
result = conn.execute(query)

for row in result:
    print(row)

2018-07-10 14:33:28,110 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 14:33:28,110 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, Kim')
(2, 'kim', 'Anonymous, Kim')
(3, 'kim', 'Anonymous, Kim')
(4, 'Lee', 'Unknown, Lee')


In [66]:
result = conn.execute(select([users.c.name, users.c.fullname]))
for row in result:
    print(row)

2018-07-10 14:40:44,936 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2018-07-10 14:40:44,937 INFO sqlalchemy.engine.base.Engine ()
('kim', 'Anonymous, Kim')
('kim', 'Anonymous, Kim')
('kim', 'Anonymous, Kim')
('Lee', 'Unknown, Lee')


In [67]:
result = conn.execute(query)
row = result.fetchone()
print('id -', row['id'], ', name -', row['name'], ', fullname -',row['fullname'])

2018-07-10 14:40:45,515 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 14:40:45,516 INFO sqlalchemy.engine.base.Engine ()
id - 1 , name - kim , fullname - Anonymous, Kim


In [68]:
row = result.fetchone()
print('id -', row[0], ', name -', row[1], ', fullname -',row[2])

id - 2 , name - kim , fullname - Anonymous, Kim


In [69]:
result = conn.execute(query)
rows = result.fetchall()
for row in rows:
    print('id -',row[0], ', name -', row[1], ', fullname -', row[2])
    
result.close()
    

2018-07-10 14:40:53,213 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 14:40:53,214 INFO sqlalchemy.engine.base.Engine ()
id - 1 , name - kim , fullname - Anonymous, Kim
id - 2 , name - kim , fullname - Anonymous, Kim
id - 3 , name - kim , fullname - Anonymous, Kim
id - 4 , name - Lee , fullname - Unknown, Lee


In [72]:
from sqlalchemy import and_, or_, not_
print(users.c.id == addresses.c.user_id)
print(users.c.id == 1)
print((users.c.id == 1).compile().params)
print(or_(users.c.id == addresses.c.user_id, users.c.id == 1))
print(and_(users.c.id == addresses.c.user_id, users.c.id == 1))

users.id = addresses.user_id
users.id = :id_1
{'id_1': 1}
users.id = addresses.user_id OR users.id = :id_1
users.id = addresses.user_id AND users.id = :id_1


In [73]:
print(and_(
        or_(
            users.c.id == addresses.c.user_id,
            users.c.id == 1
        ),
        addresses.c.email_address.like('a%')
        )
    )

print((
        (users.c.id == addresses.c.user_id) |
        (users.c.id == 1)
    ) & (addresses.c.email_address.like('a%')))

(users.id = addresses.user_id OR users.id = :id_1) AND addresses.email_address LIKE :email_address_1
(users.id = addresses.user_id OR users.id = :id_1) AND addresses.email_address LIKE :email_address_1


In [74]:
result = conn.execute(select([users]).where(users.c.id==1))
for row in result:
    print(row)

2018-07-10 15:09:36,055 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2018-07-10 15:09:36,056 INFO sqlalchemy.engine.base.Engine (1,)
(1, 'kim', 'Anonymous, Kim')


In [75]:
from sqlalchemy import join
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


In [76]:
print(users.join(addresses, users.c.id==addresses.c.user_id))

users JOIN addresses ON users.id = addresses.user_id


In [77]:
result = conn.execute(
                select([users.c.name, addresses.c.email_address])
                .select_from(users.join(addresses)))

2018-07-10 15:13:46,309 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2018-07-10 15:13:46,310 INFO sqlalchemy.engine.base.Engine ()


In [79]:
rows = result.fetchall()
for row in rows:
    print(row)

('kim', 'anonymous.kim@test.com')
('kim', 'unknow.lee@test.com')


In [82]:
artist = Table('Artist',meta,
               Column('id', Integer, primary_key=True),
               Column('name', String, nullable=False),
               extend_existing=True)

album = Table('Album',meta,
               Column('id', Integer, primary_key=True),
               Column('title', String, nullable=False),
               Column('artist_id', Integer, ForeignKey('Artist.id')),
               extend_existing=True)

genre = Table('Genre',meta,
               Column('id', Integer, primary_key=True),
               Column('name', String, nullable=False),
               extend_existing=True)

track = Table('Track',meta,
               Column('id', Integer, primary_key=True),
               Column('title', String, nullable=False),
               Column('length', Integer, nullable=False),
               Column('rating', Integer, nullable=False),
               Column('count', Integer, nullable=False),
               Column('album_id', Integer, ForeignKey('Album.id')),
               Column('genre_id', Integer, ForeignKey('Genre.id')),
               extend_existing=True)

meta.create_all(engine)

2018-07-10 15:21:38,836 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-10 15:21:38,836 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:21:38,837 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-07-10 15:21:38,837 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:21:38,838 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2018-07-10 15:21:38,838 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:21:38,840 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-10 15:21:38,840 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:21:38,841 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2018-07-10 15:21:38,841 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:21:38,842 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2018-07-10 15:21:38,843 INFO sqlalchemy.engine.base.Engine ()


In [94]:
users.drop(engine)
addresses.drop(engine)

for row in meta.tables:
    print(row)
for row in engine.table_names():
    print(row)

2018-07-10 15:27:10,971 INFO sqlalchemy.engine.base.Engine 
DROP TABLE users
2018-07-10 15:27:10,972 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 15:27:10,972 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: users [SQL: '\nDROP TABLE users'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [95]:
conn.execute(artist.insert(), [
    {'name':'Led Zepplin'},
    {'name':'AC/DC'}  
])

conn.execute(album.insert(), [
    {'title':'IV', 'artist_id':1},
    {'title':'Who Made Who', "artist_id":2}  
])

conn.execute(genre.insert(), [
    {'name':'Rock'},
    {'name':'Metal'}  
])

conn.execute(track.insert(), [
    {'title':'Black Dog','rating':5, 'length':297, 'count':0, 'album_id':1, 'genre_id':1},
    {'title':'Stairway','rating':5, 'length':482, 'count':0, 'album_id':1, 'genre_id':1},
    {'title':'About to rock','rating':5, 'length':313, 'count':0, 'album_id':2, 'genre_id':2},
    {'title':'Who Made Who','rating':5, 'length':297, 'count':0, 'album_id':2, 'genre_id':2},
])

2018-07-10 15:32:58,412 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2018-07-10 15:32:58,413 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))
2018-07-10 15:32:58,415 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 15:32:58,421 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2018-07-10 15:32:58,422 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('Who Made Who', 2))
2018-07-10 15:32:58,423 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 15:32:58,428 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2018-07-10 15:32:58,428 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2018-07-10 15:32:58,430 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 15:32:58,437 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2018-07-10 15:32:58,438 INFO sqlalchemy.engine.base.Engine (('Black Dog', 

In [98]:
artistResult = conn.execute(artist.select())
for row in artistResult:
    print(row)

2018-07-10 15:36:22,022 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id, "Artist".name 
FROM "Artist"
2018-07-10 15:36:22,023 INFO sqlalchemy.engine.base.Engine ()
(1, 'Led Zepplin')
(2, 'AC/DC')


In [97]:
albumResult = conn.execute(album.select())
for row in albumResult:
    print(row)

2018-07-10 15:36:10,051 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2018-07-10 15:36:10,052 INFO sqlalchemy.engine.base.Engine ()
(1, 'IV', 1)
(2, 'Who Made Who', 2)


In [99]:
genreResult = conn.execute(genre.select())
for row in genreResult:
    print(row)

2018-07-10 15:37:00,373 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id, "Genre".name 
FROM "Genre"
2018-07-10 15:37:00,374 INFO sqlalchemy.engine.base.Engine ()
(1, 'Rock')
(2, 'Metal')


In [106]:
trackResult = conn.execute(track.select())
for row in trackResult:
    print(row)

2018-07-10 15:41:30,808 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"
2018-07-10 15:41:30,809 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)
(3, 'About to rock', 313, 5, 0, 2, 2)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [111]:
trackResult = conn.execute(select([track])
                     .where(
                         and_(track.c.album_id == 1,
                              or_(track.c.genre_id == 1,
                                  track.c.genre_id == 2)
                             )
                         )) 
for row in trackResult:
    print(row)

2018-07-10 15:43:48,144 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2018-07-10 15:43:48,145 INFO sqlalchemy.engine.base.Engine (1, 1, 2)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)


In [112]:
from sqlalchemy import update

conn.execute(track.update().values(genre_id = 2)
             .where(track.c.id == 2))
conn.execute(track.update().values(genre_id = 1)
             .where(track.c.id == 3))

2018-07-10 15:45:20,336 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-10 15:45:20,338 INFO sqlalchemy.engine.base.Engine (2, 2)
2018-07-10 15:45:20,339 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 15:45:20,347 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-10 15:45:20,348 INFO sqlalchemy.engine.base.Engine (1, 3)
2018-07-10 15:45:20,349 INFO sqlalchemy.engine.base.Engine COMMIT


In [113]:
trackResult = conn.execute(select([track])
                     .where(
                         and_(track.c.album_id == 1,
                              or_(track.c.genre_id == 1,
                                  track.c.genre_id == 2)
                             )
                         )) 
for row in trackResult:
    print(row)

2018-07-10 15:46:52,213 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2018-07-10 15:46:52,214 INFO sqlalchemy.engine.base.Engine (1, 1, 2)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 2)


In [114]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [115]:
result = conn.execute(track.select()
             .select_from(track.join(album)))
for row in result:
    print(row)

2018-07-10 15:48:49,672 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2018-07-10 15:48:49,673 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 2)
(3, 'About to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [117]:
result = conn.execute(select([track.c.title, album.c.title]).select_from(track.join(album)))
for row in result:
    print(row)

2018-07-10 15:51:14,361 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2018-07-10 15:51:14,362 INFO sqlalchemy.engine.base.Engine ()
('Black Dog', 'IV')
('Stairway', 'IV')
('About to rock', 'Who Made Who')
('Who Made Who', 'Who Made Who')
